In [1]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/envs')
sys.path.append('/models')
import envs.viper as vpr
import models.models as models
import models.dqn as dqn

In [3]:
import importlib
importlib.reload(vpr); importlib.reload(models); importlib.reload(dqn)

<module 'models.dqn' from 'c:\\Users\\Nickolas\\Documents\\GitHub\\Deep-XIV-Rotation\\models\\dqn.py'>

In [4]:
viper_env = vpr.Viper(_sks = 798)

b123


In [5]:
torch.cuda.empty_cache()

In [6]:
agent = dqn.DQN(viper_env, _max_history=2048)

Created model dense_v1 with 15 features and 9 actions.
Model loaded onto cpu.
Layer (type:depth-idx)                   Output Shape              Param #
DenseNetV3                               [196, 9]                  --
├─Sequential: 1-1                        [196, 9]                  --
│    └─Linear: 2-1                       [196, 4096]               65,536
│    └─SELU: 2-2                         [196, 4096]               --
│    └─Linear: 2-3                       [196, 4096]               16,781,312
│    └─Tanh: 2-4                         [196, 4096]               --
│    └─Linear: 2-5                       [196, 2048]               8,390,656
│    └─BatchNorm1d: 2-6                  [196, 2048]               4,096
│    └─ReLU: 2-7                         [196, 2048]               --
│    └─Dropout: 2-8                      [196, 2048]               --
│    └─Linear: 2-9                       [196, 1024]               2,098,176
│    └─ReLU: 2-10                        [196, 1

In [7]:
action_sequence=[1, 3, 6, 8, 0, 2, 5, 0, 0]
agent.env.reset_env()
_sum = 0
for a in action_sequence:
    res = agent.env.step(a, True)
    _sum+=res[0]
    print(f'({res[0]:.2f}, {res[1]:.0f}, {res[2]:.2f})')
print(_sum)

Took action: 1-reaving_fangs @ 0.000
(0.43, 200, 206.13)
Took action: 3-swiftskins_sting @ 2.450
(0.61, 300, 286.20)
Took action: 6-hindsting_strike @ 4.900
(0.83, 400, 384.62)
Took action: 8-death_rattle @ 5.600
(0.61, 280, 274.43)
Took action: 0-steel_fangs @ 6.980
(0.64, 300, 306.96)
Took action: 2-hunters_sting @ 9.060
(0.62, 300, 296.72)
Took action: 5-flanksbane_fang @ 11.140
(1.18, 550, 527.77)
Took action: 0-steel_fangs @ 13.220
(0.44, 220, 221.38)
Took action: 0-steel_fangs @ 13.920
(-0.23, 0, 0.00)
5.126222222222222


In [8]:
agent.env.reset_env()
print(agent.env.state())
print(agent.env.valid_actions())
print([np.random.choice(agent.env.valid_actions()[1])])
print(agent.env.gcd)

tensor([ 2.4500, 10.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000])
([True, True, False, False, False, False, False, False, False], [0, 1])
[1]
2.45


In [9]:
agent.get_action(agent.env.state(), e=1)[0]

tensor(7, dtype=torch.int32)

In [10]:
agent.train(gamma=0.80, num_epochs=5, session_limit=100)

Epoch 0 Loss: 3.28e-04 E_0: 0.25 E_1: 0.10 G: 0.80 Rewards: -5.5 Eval Rewards: -11.56, LR: 1.3e-05 SKS: 768
Epoch 1 Loss: 1.13e-04 E_0: 0.25 E_1: 0.10 G: 0.80 Rewards: -8.4 Eval Rewards: -11.56, LR: 1.3e-05 SKS: 528
Epoch 2 Loss: 8.73e-05 E_0: 0.25 E_1: 0.10 G: 0.80 Rewards: -4.3 Eval Rewards: -11.56, LR: 1.3e-05 SKS: 582


KeyboardInterrupt: 

In [ ]:
history_x = agent.training_history_x
history_y = agent.training_history_y
fig, ax = plt.subplots()
fig.suptitle(f'Rewards {agent.model_name} LR: {agent.lr:.1e}')
ax.plot(history_x, history_y)
ax.legend(['Train Rewards', 'Eval Rewards'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Rewards')
plt.savefig('rewards_graph.png', bbox_inches='tight')
plt.show()

In [11]:
agent.load_checkpoint(f'./checkpoints/_{agent.model_name}_best.pth')

In [ ]:
viper_env.compute_gcd(2.5, 798, 15)

In [ ]:
# 2.12, 2.11, 2.10, 2.09, 2.08
#[420, 528, 582, 768, 822]
agent.test(num_steps=50, _sks=420)

In [ ]:
pred = agent.predict(agent.env.state())
pred

In [ ]:
agent.env.reset_env()
pred = agent.predict(agent.env.state())
pred

In [ ]:
agent.env.state()

terraform aws
kubernites
helm?
look into